In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import torchvision
import torch
from PIL import Image
import torchvision.transforms as T
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
import math

## **bounding box 좌표 찾고 crop해서 내보내는 코드**

In [ ]:
# 미리 학습된 Mask R-CNN 모델 로드
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

# 모델을 평가 모드로 설정
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:02<00:00, 68.8MB/s]


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
# bounding box가 여러 개가 나올 수 있으므로, 확률 값이 가장 큰 bounding box의 index 값을 추출하는 함수 정의
def find_max_index(tensor):
    max_value, max_index = torch.max(tensor, dim=0)
    return max_index.item()

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/캐주얼/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
for i in range(len(mall)):
  path2 = path + mall[i] + '/'
  file_list = os.listdir(path2)
  file_list_py = [file for file in file_list if file.endswith('.jpg')] # 한 쇼핑몰 폴더에 있는 이미지 파일 리스트

  for j in range(len(file_list_py)) :
    # 입력 이미지 로드
    image = Image.open(path2 + file_list_py[j])
    # 이미지를 Torch Tensor로 변환
    transform = T.Compose([T.ToTensor()])
    input_image = transform(image)
    # 이미지 배치(batch)로 변환
    input_image = input_image.unsqueeze(0)

    # 모델 예측 수행
    with torch.no_grad():
      prediction = model(input_image)

    boxes = prediction[0]['boxes'] # bounding box 좌표
    scores = prediction[0]['scores'] #객체의 확률 값들

    # 확률 값이 가장 큰 bounding box 좌표 출력
    bounding_box = boxes[find_max_index(scores)]

    # 텐서에서 바운딩 박스 좌표 얻기
    x_min, y_min, x_max, y_max = bounding_box

    # 텐서를 정수로 반환(인덱싱 하려고)
    x_min = math.floor(x_min.item())
    y_min = math.floor(y_min.item())
    x_max = math.floor(x_max.item())
    y_max = math.floor(y_max.item())

    # 이미지 읽기
    image = cv2.imread(path2 + file_list_py[j])

    # 바운딩 박스 좌표를 이용하여 이미지 crop
    cropped_image = image[y_min:y_max, x_min:x_max]

    # crop된 이미지 저장 경로
    output_path = '/content/drive/MyDrive/데청캠_공유/쇼핑몰 크롭/캐주얼/' + mall[i] + '/cropped_' + file_list_py[j]

    # crop된 이미지 저장
    cv2.imwrite(output_path, cropped_image)

    print(f'{mall[i]}의{j}번째 이미지가 성공적으로 저장되었습니다.')

니썸의0번째 이미지가 성공적으로 저장되었습니다.
니썸의1번째 이미지가 성공적으로 저장되었습니다.
니썸의2번째 이미지가 성공적으로 저장되었습니다.
니썸의3번째 이미지가 성공적으로 저장되었습니다.
니썸의4번째 이미지가 성공적으로 저장되었습니다.
니썸의5번째 이미지가 성공적으로 저장되었습니다.
니썸의6번째 이미지가 성공적으로 저장되었습니다.
니썸의7번째 이미지가 성공적으로 저장되었습니다.
니썸의8번째 이미지가 성공적으로 저장되었습니다.
니썸의9번째 이미지가 성공적으로 저장되었습니다.
니썸의10번째 이미지가 성공적으로 저장되었습니다.
니썸의11번째 이미지가 성공적으로 저장되었습니다.
니썸의12번째 이미지가 성공적으로 저장되었습니다.
니썸의13번째 이미지가 성공적으로 저장되었습니다.
니썸의14번째 이미지가 성공적으로 저장되었습니다.
니썸의15번째 이미지가 성공적으로 저장되었습니다.
니썸의16번째 이미지가 성공적으로 저장되었습니다.
니썸의17번째 이미지가 성공적으로 저장되었습니다.
니썸의18번째 이미지가 성공적으로 저장되었습니다.
니썸의19번째 이미지가 성공적으로 저장되었습니다.
니썸의20번째 이미지가 성공적으로 저장되었습니다.
니썸의21번째 이미지가 성공적으로 저장되었습니다.
니썸의22번째 이미지가 성공적으로 저장되었습니다.
니썸의23번째 이미지가 성공적으로 저장되었습니다.
니썸의24번째 이미지가 성공적으로 저장되었습니다.
니썸의25번째 이미지가 성공적으로 저장되었습니다.
니썸의26번째 이미지가 성공적으로 저장되었습니다.
니썸의27번째 이미지가 성공적으로 저장되었습니다.
니썸의28번째 이미지가 성공적으로 저장되었습니다.
니썸의29번째 이미지가 성공적으로 저장되었습니다.
필링스의0번째 이미지가 성공적으로 저장되었습니다.
필링스의1번째 이미지가 성공적으로 저장되었습니다.
필링스의2번째 이

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/캠퍼스룩/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
for i in range(len(mall)):
  path2 = path + mall[i] + '/'
  file_list = os.listdir(path2)
  file_list_py = [file for file in file_list if file.endswith('.jpg')] # 한 쇼핑몰 폴더에 있는 이미지 파일 리스트

  for j in range(len(file_list_py)) :
    # 입력 이미지 로드
    image = Image.open(path2 + file_list_py[j])
    # 이미지를 Torch Tensor로 변환
    transform = T.Compose([T.ToTensor()])
    input_image = transform(image)
    # 이미지 배치(batch)로 변환
    input_image = input_image.unsqueeze(0)

    # 모델 예측 수행
    with torch.no_grad():
      prediction = model(input_image)

    boxes = prediction[0]['boxes'] # bounding box 좌표
    scores = prediction[0]['scores'] #객체의 확률 값들

    # 확률 값이 가장 큰 bounding box 좌표 출력
    bounding_box = boxes[find_max_index(scores)]

    # 텐서에서 바운딩 박스 좌표 얻기
    x_min, y_min, x_max, y_max = bounding_box

    # 텐서를 정수로 반환(인덱싱 하려고)
    x_min = math.floor(x_min.item())
    y_min = math.floor(y_min.item())
    x_max = math.floor(x_max.item())
    y_max = math.floor(y_max.item())

    # 이미지 읽기
    image = cv2.imread(path2 + file_list_py[j])

    # 바운딩 박스 좌표를 이용하여 이미지 crop
    cropped_image = image[y_min:y_max, x_min:x_max]

    # crop된 이미지 저장 경로
    output_path = '/content/drive/MyDrive/데청캠_공유/쇼핑몰 크롭/캠퍼스룩/' + mall[i] + '/cropped_' + file_list_py[j]

    # crop된 이미지 저장
    cv2.imwrite(output_path, cropped_image)

    print(f'{i}번째 쇼핑몰인 {mall[i]}의 {j}번째 이미지가 성공적으로 저장되었습니다.')

0번째 쇼핑몰인 위키드클로젯의 0번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 1번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 2번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 3번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 4번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 5번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 6번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 7번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 8번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 9번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 10번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 11번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 12번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 13번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 14번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 15번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 16번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 17번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 18번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 위키드클로젯의 19번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 ᄋ

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/오피스룩/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
for i in range(len(mall)):
  path2 = path + mall[i] + '/'
  file_list = os.listdir(path2)
  file_list_py = [file for file in file_list if file.endswith('.jpg')] # 한 쇼핑몰 폴더에 있는 이미지 파일 리스트

  for j in range(len(file_list_py)) :
    # 입력 이미지 로드
    image = Image.open(path2 + file_list_py[j])
    # 이미지를 Torch Tensor로 변환
    transform = T.Compose([T.ToTensor()])
    input_image = transform(image)
    # 이미지 배치(batch)로 변환
    input_image = input_image.unsqueeze(0)

    # 모델 예측 수행
    with torch.no_grad():
      prediction = model(input_image)

    boxes = prediction[0]['boxes'] # bounding box 좌표
    scores = prediction[0]['scores'] #객체의 확률 값들

    # 확률 값이 가장 큰 bounding box 좌표 출력
    bounding_box = boxes[find_max_index(scores)]

    # 텐서에서 바운딩 박스 좌표 얻기
    x_min, y_min, x_max, y_max = bounding_box

    # 텐서를 정수로 반환(인덱싱 하려고)
    x_min = math.floor(x_min.item())
    y_min = math.floor(y_min.item())
    x_max = math.floor(x_max.item())
    y_max = math.floor(y_max.item())

    # 이미지 읽기
    image = cv2.imread(path2 + file_list_py[j])

    # 바운딩 박스 좌표를 이용하여 이미지 crop
    cropped_image = image[y_min:y_max, x_min:x_max]

    # crop된 이미지 저장 경로
    output_path = '/content/drive/MyDrive/데청캠_공유/쇼핑몰 크롭/오피스룩/' + mall[i] + '/cropped_' + file_list_py[j]

    # crop된 이미지 저장
    cv2.imwrite(output_path, cropped_image)

    print(f'{i}번째 쇼핑몰인 {mall[i]}의 {j}번째 이미지가 성공적으로 저장되었습니다.')

0번째 쇼핑몰인 베니토의 0번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 1번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 2번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 3번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 4번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 5번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 6번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 7번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 8번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 9번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 10번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 11번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 12번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 13번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 14번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 15번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 16번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 17번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 18번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 19번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 20번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 21번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 22번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 베니토의 23번째 이미지가 성공적으로 저장되었습니다.
0번

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/빈티지/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
for i in range(len(mall)):
  path2 = path + mall[i] + '/'
  file_list = os.listdir(path2)
  file_list_py = [file for file in file_list if file.endswith('.jpg')] # 한 쇼핑몰 폴더에 있는 이미지 파일 리스트

  for j in range(len(file_list_py)) :
    # 입력 이미지 로드
    image = Image.open(path2 + file_list_py[j])
    # 이미지를 Torch Tensor로 변환
    transform = T.Compose([T.ToTensor()])
    input_image = transform(image)
    # 이미지 배치(batch)로 변환
    input_image = input_image.unsqueeze(0)

    # 모델 예측 수행
    with torch.no_grad():
      prediction = model(input_image)

    boxes = prediction[0]['boxes'] # bounding box 좌표
    scores = prediction[0]['scores'] #객체의 확률 값들

    # 확률 값이 가장 큰 bounding box 좌표 출력
    bounding_box = boxes[find_max_index(scores)]

    # 텐서에서 바운딩 박스 좌표 얻기
    x_min, y_min, x_max, y_max = bounding_box

    # 텐서를 정수로 반환(인덱싱 하려고)
    x_min = math.floor(x_min.item())
    y_min = math.floor(y_min.item())
    x_max = math.floor(x_max.item())
    y_max = math.floor(y_max.item())

    # 이미지 읽기
    image = cv2.imread(path2 + file_list_py[j])

    # 바운딩 박스 좌표를 이용하여 이미지 crop
    cropped_image = image[y_min:y_max, x_min:x_max]

    # crop된 이미지 저장 경로
    output_path = '/content/drive/MyDrive/데청캠_공유/쇼핑몰 크롭/빈티지/' + mall[i] + '/cropped_' + file_list_py[j]

    # crop된 이미지 저장
    cv2.imwrite(output_path, cropped_image)

    print(f'{i}번째 쇼핑몰인 {mall[i]}의 {j}번째 이미지가 성공적으로 저장되었습니다.')

0번째 쇼핑몰인 모래하우스의 0번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 1번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 2번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 3번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 4번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 5번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 6번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 7번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 8번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 9번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 10번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 11번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 12번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 13번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 14번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 15번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 16번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 17번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 18번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 19번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 20번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 모래하우스의 21번째 이미지가 성공적으로 저장되었습니다

TypeError: ignored

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/빈티지/프라이빗룩북/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

for i in range(20,len(file_list_py)):

  # 입력 이미지 로드
  image = Image.open(path + file_list_py[i])
  # 이미지를 Torch Tensor로 변환
  transform = T.Compose([T.ToTensor()])
  input_image = transform(image)
  # 이미지 배치(batch)로 변환
  input_image = input_image.unsqueeze(0)

  # 모델 예측 수행
  with torch.no_grad():
    prediction = model(input_image)

  boxes = prediction[0]['boxes'] # bounding box 좌표
  scores = prediction[0]['scores'] #객체의 확률 값들

  # 확률 값이 가장 큰 bounding box 좌표 출력
  bounding_box = boxes[find_max_index(scores)]

  # 텐서에서 바운딩 박스 좌표 얻기
  x_min, y_min, x_max, y_max = bounding_box

  # 텐서를 정수로 반환(인덱싱 하려고)
  x_min = math.floor(x_min.item())
  y_min = math.floor(y_min.item())
  x_max = math.floor(x_max.item())
  y_max = math.floor(y_max.item())

  # 이미지 읽기
  image = cv2.imread(path + file_list_py[i])

  # 바운딩 박스 좌표를 이용하여 이미지 crop
  cropped_image = image[y_min:y_max, x_min:x_max]

  # crop된 이미지 저장 경로
  output_path = '/content/drive/MyDrive/데청캠_공유/쇼핑몰 크롭/빈티지/프라이빗룩북/' + 'cropped_' + file_list_py[i]

  # crop된 이미지 저장
  cv2.imwrite(output_path, cropped_image)

  print(f'{i}번째 이미지가 성공적으로 저장되었습니다.')


20번째 이미지가 성공적으로 저장되었습니다.
21번째 이미지가 성공적으로 저장되었습니다.
22번째 이미지가 성공적으로 저장되었습니다.
23번째 이미지가 성공적으로 저장되었습니다.
24번째 이미지가 성공적으로 저장되었습니다.
25번째 이미지가 성공적으로 저장되었습니다.
26번째 이미지가 성공적으로 저장되었습니다.
27번째 이미지가 성공적으로 저장되었습니다.
28번째 이미지가 성공적으로 저장되었습니다.
29번째 이미지가 성공적으로 저장되었습니다.


In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/빈티지/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
for i in range(5,len(mall)):
  path2 = path + mall[i] + '/'
  file_list = os.listdir(path2)
  file_list_py = [file for file in file_list if file.endswith('.jpg')] # 한 쇼핑몰 폴더에 있는 이미지 파일 리스트

  for j in range(len(file_list_py)) :
    # 입력 이미지 로드
    image = Image.open(path2 + file_list_py[j])
    # 이미지를 Torch Tensor로 변환
    transform = T.Compose([T.ToTensor()])
    input_image = transform(image)
    # 이미지 배치(batch)로 변환
    input_image = input_image.unsqueeze(0)

    # 모델 예측 수행
    with torch.no_grad():
      prediction = model(input_image)

    boxes = prediction[0]['boxes'] # bounding box 좌표
    scores = prediction[0]['scores'] #객체의 확률 값들

    # 확률 값이 가장 큰 bounding box 좌표 출력
    bounding_box = boxes[find_max_index(scores)]

    # 텐서에서 바운딩 박스 좌표 얻기
    x_min, y_min, x_max, y_max = bounding_box

    # 텐서를 정수로 반환(인덱싱 하려고)
    x_min = math.floor(x_min.item())
    y_min = math.floor(y_min.item())
    x_max = math.floor(x_max.item())
    y_max = math.floor(y_max.item())

    # 이미지 읽기
    image = cv2.imread(path2 + file_list_py[j])

    # 바운딩 박스 좌표를 이용하여 이미지 crop
    cropped_image = image[y_min:y_max, x_min:x_max]

    # crop된 이미지 저장 경로
    output_path = '/content/drive/MyDrive/데청캠_공유/쇼핑몰 크롭/빈티지/' + mall[i] + '/cropped_' + file_list_py[j]

    # crop된 이미지 저장
    cv2.imwrite(output_path, cropped_image)

    print(f'{i}번째 쇼핑몰인 {mall[i]}의 {j}번째 이미지가 성공적으로 저장되었습니다.')

5번째 쇼핑몰인 워무드마인의 0번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 1번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 2번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 3번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 4번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 5번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 6번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 7번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 8번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 9번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 10번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 11번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 12번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 13번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 14번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 15번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 16번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 17번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 18번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 19번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 20번째 이미지가 성공적으로 저장되었습니다.
5번째 쇼핑몰인 워무드마인의 2

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/유니크/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
for i in range(len(mall)):
  path2 = path + mall[i] + '/'
  file_list = os.listdir(path2)
  file_list_py = [file for file in file_list if file.endswith('.jpg')] # 한 쇼핑몰 폴더에 있는 이미지 파일 리스트

  for j in range(len(file_list_py)) :
    # 입력 이미지 로드
    image = Image.open(path2 + file_list_py[j])
    # 이미지를 Torch Tensor로 변환
    transform = T.Compose([T.ToTensor()])
    input_image = transform(image)
    # 이미지 배치(batch)로 변환
    input_image = input_image.unsqueeze(0)

    # 모델 예측 수행
    with torch.no_grad():
      prediction = model(input_image)

    boxes = prediction[0]['boxes'] # bounding box 좌표
    scores = prediction[0]['scores'] #객체의 확률 값들

    # 확률 값이 가장 큰 bounding box 좌표 출력
    bounding_box = boxes[find_max_index(scores)]

    # 텐서에서 바운딩 박스 좌표 얻기
    x_min, y_min, x_max, y_max = bounding_box

    # 텐서를 정수로 반환(인덱싱 하려고)
    x_min = math.floor(x_min.item())
    y_min = math.floor(y_min.item())
    x_max = math.floor(x_max.item())
    y_max = math.floor(y_max.item())

    # 이미지 읽기
    image = cv2.imread(path2 + file_list_py[j])

    # 바운딩 박스 좌표를 이용하여 이미지 crop
    cropped_image = image[y_min:y_max, x_min:x_max]

    # crop된 이미지 저장 경로
    output_path = '/content/drive/MyDrive/데청캠_공유/쇼핑몰 크롭/유니크/' + mall[i] + '/cropped_' + file_list_py[j]

    # crop된 이미지 저장
    cv2.imwrite(output_path, cropped_image)

    print(f'{i}번째 쇼핑몰인 {mall[i]}의 {j}번째 이미지가 성공적으로 저장되었습니다.')

0번째 쇼핑몰인 하비언니의 0번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 1번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 2번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 3번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 4번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 5번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 6번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 7번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 8번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 9번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 10번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 11번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 12번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 13번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 14번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 15번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 16번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 17번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 18번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 19번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 20번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 21번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 하비언니의 

In [ ]:
# 이미지 목록을 불러오기
path = "/content/drive/MyDrive/데청캠_공유/쇼핑몰/심플베이직/" # 스타일 입력
mall = os.listdir(path) # 쇼핑몰 리스트
for i in range(len(mall)):
  path2 = path + mall[i] + '/'
  file_list = os.listdir(path2)
  file_list_py = [file for file in file_list if file.endswith('.jpg')] # 한 쇼핑몰 폴더에 있는 이미지 파일 리스트

  for j in range(len(file_list_py)) :
    # 입력 이미지 로드
    image = Image.open(path2 + file_list_py[j])
    # 이미지를 Torch Tensor로 변환
    transform = T.Compose([T.ToTensor()])
    input_image = transform(image)
    # 이미지 배치(batch)로 변환
    input_image = input_image.unsqueeze(0)

    # 모델 예측 수행
    with torch.no_grad():
      prediction = model(input_image)

    boxes = prediction[0]['boxes'] # bounding box 좌표
    scores = prediction[0]['scores'] #객체의 확률 값들

    # 확률 값이 가장 큰 bounding box 좌표 출력
    bounding_box = boxes[find_max_index(scores)]

    # 텐서에서 바운딩 박스 좌표 얻기
    x_min, y_min, x_max, y_max = bounding_box

    # 텐서를 정수로 반환(인덱싱 하려고)
    x_min = math.floor(x_min.item())
    y_min = math.floor(y_min.item())
    x_max = math.floor(x_max.item())
    y_max = math.floor(y_max.item())

    # 이미지 읽기
    image = cv2.imread(path2 + file_list_py[j])

    # 바운딩 박스 좌표를 이용하여 이미지 crop
    cropped_image = image[y_min:y_max, x_min:x_max]

    # crop된 이미지 저장 경로
    output_path = '/content/drive/MyDrive/데청캠_공유/쇼핑몰 크롭/심플베이직/' + mall[i] + '/cropped_' + file_list_py[j]

    # crop된 이미지 저장
    cv2.imwrite(output_path, cropped_image)

    print(f'{i}번째 쇼핑몰인 {mall[i]}의 {j}번째 이미지가 성공적으로 저장되었습니다.')

0번째 쇼핑몰인 아위의 0번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 1번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 2번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 3번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 4번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 5번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 6번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 7번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 8번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 9번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 10번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 11번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 12번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 13번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 14번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 15번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 16번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 17번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 18번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 19번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 20번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 21번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 22번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 23번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 아위의 24번째 이미지가 성공적으로 저장되었습니다.
0번째 쇼핑몰인 ᄋ